In [1]:
import pandas as pd
import numpy as np

In [2]:
visits_url = 'http://rocker-data-engineering-task.storage.googleapis.com/data/visits.csv'
customers_url = 'http://rocker-data-engineering-task.storage.googleapis.com/data/customers.json'
loan_csv_list_url = 'http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2017-10.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2017-11.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2017-12.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-1.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-10.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-11.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-12.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-2.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-3.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-4.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-5.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-6.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-7.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-8.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2018-9.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-1.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-10.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-2.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-3.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-4.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-5.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-6.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-7.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-8.csv, http://rocker-data-engineering-task.storage.googleapis.com/data/loan-2019-9.csv'

### Read and preprocess

In [3]:
def download_data(loan_csv_list_url, customers_url, visits_url):
    visits = pd.read_csv(visits_url)
    customers = pd.read_json(customers_url, lines=True)
    #list of all data files, this would be easier to pick if directly from a GCS bucket
    loan_csv_list = loan_csv_list_url.split(', ')
    #for url in loan_csv_list:
    #    print(url)
    
    return loan_csv_list, customers, visits

    

def preprocess_data(loan_csv_list, customers, visits):
    #combine multiple dataframes and use datetime instead of epoch
    dfs = [pd.read_csv(url) for url in loan_csv_list]
    loan = pd.concat(dfs, ignore_index=True)
    loan['timestamp'] = pd.to_datetime(loan['timestamp'], unit='s')
    ##format webvisit_id
    loan['webvisit_id'] = loan['webvisit_id'].str.replace(r'[()]', '').str.replace(r',', '')
    #webvisit_id object to numeric
    loan['webvisit_id'] = pd.to_numeric(loan['webvisit_id'], errors='coerce').convert_dtypes() 
    loan['webvisit_id'] =loan['webvisit_id'].fillna(0)
    loan = loan.drop("Unnamed: 0",axis=1)
    loan = loan.replace(0, np.nan)
    
    #use datetime
    visits = visits.drop("Unnamed: 0",axis=1)
    visits['timestamp'] = pd.to_datetime(visits['timestamp'], unit='s')
    
    return loan, visits

In [4]:
loan_csv_list, customers, visits = download_data(loan_csv_list_url, customers_url, visits_url)
loan, visits = preprocess_data(loan_csv_list, customers, visits)

In [5]:
loan.dtypes
#loan.isnull().sum()

id                       int64
user_id                  int64
timestamp       datetime64[ns]
loan_amount              int64
loan_purpose            object
outcome                 object
interest               float64
webvisit_id              Int64
dtype: object

In [6]:
customers.dtypes

id           int64
name        object
ssn         object
birthday    object
gender      object
city        object
zip_code    object
dtype: object

In [7]:
visits.dtypes

id                        int64
timestamp        datetime64[ns]
referrer                 object
campaign_name            object
dtype: object

### lets look at the data

In [8]:
loan.head()

,id,user_id,timestamp,loan_amount,loan_purpose,outcome,interest,webvisit_id
0,9546870,33593540,2017-10-28 03:42:29,80000,Buying a pet,ACCEPTED,8.7,<NA>
1,36469880,41352735,2017-10-24 23:35:34,270000,Refinance existing loans,REJECTED,6.7,72572646
2,22554526,70055796,2017-10-03 20:59:49,240000,Home purchase,ACCEPTED,7.0,54404642
3,27906548,17577009,2017-10-22 08:23:47,190000,Buying a pet,REJECTED,14.0,<NA>
4,46601919,15988868,2017-10-21 11:53:43,20000,Home purchase,REJECTED,14.0,<NA>


In [9]:
customers.head()

,id,name,ssn,birthday,gender,city,zip_code
0,20427847,Μάριος Γαρουφαλής,172-10-3586,1978-05-09,F,Γρεβενά,ΤΚ 763 78
1,74075652,Μόσχα Διαμαντοπούλου,140-57-5668,1994-03-05,F,Κομοτηνή,75665
2,31170608,Κλυταιμνήστρα Πέτση,548-79-9954,1971-08-24,M,Φλώρινα,ΤΚ 49056
3,39640871,Ευγενία Στρατογιάννη,036-43-6966,1988-12-19,M,Φλώρινα,65934
4,60718455,Χαρίτος-Τίμων Τσατσάνης,467-48-0823,1989-07-21,F,Θεσσαλονίκη,ΤΚ 38335


In [10]:
visits.head()
#dfa =visits.pivot_table(index=['id'], aggfunc='size')

,id,timestamp,referrer,campaign_name
0,80139861,2018-08-07 22:30:13,Twitter,display1
1,80139861,2017-11-11 03:32:59,Facebook,display1
2,32212575,2018-04-11 03:23:08,Google,display2
3,32212575,2017-12-03 20:00:18,Facebook,display3
4,32212575,2018-09-23 20:23:40,Google,display2


### merge all csv files

In [11]:
#loan.isnull().sum()

#dropna() is used as a workaround to exclude left join on nan values
loan_visits_ljoin = pd.merge(loan[pd.notnull(loan.webvisit_id)], visits, how='left', left_on='webvisit_id', right_on='id',  suffixes=('_loan', '_visits'))
loan_visits_ljoin.sort_values('user_id').head()


,id_loan,user_id,timestamp_loan,loan_amount,loan_purpose,outcome,interest,webvisit_id,id_visits,timestamp_visits,referrer,campaign_name
859,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,54354971.0,2019-06-30 04:35:18,Facebook,display2
858,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,54354971.0,2019-01-19 07:51:44,Google,display2
857,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,54354971.0,2018-11-25 01:07:37,Google,display3
856,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,54354971.0,2018-12-18 14:27:07,Facebook,display3
2857,10408516,49747,2018-12-19 01:06:35,220000,Buying a pet,REJECTED,6.7,3833218,NaN,NaT,NaN,NaN


In [12]:
loan_customers_ljoin = pd.merge(loan, customers, how='left', left_on='user_id', right_on='id', suffixes=('_loan', '_customers'))
loan_customers_ljoin.sort_values('user_id').head()

,id_loan,user_id,timestamp,loan_amount,loan_purpose,outcome,interest,webvisit_id,id_customers,name,ssn,birthday,gender,city,zip_code
6336,89754689,29817,2018-05-06 20:42:40,290000,Home purchase,ACCEPTED,8.7,<NA>,29817,Νεοκλής Γιαννακέας,539-03-6064,1995-08-10,M,Φλώρινα,17331
1143,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,46037,Ελισσαίος Κουτσικόπουλος,219-88-6096,1996-08-05,M,Κιλκίς,ΤΚ 464 00
3773,10408516,49747,2018-12-19 01:06:35,220000,Buying a pet,REJECTED,6.7,3833218,49747,Ευθαλία-Λεμονιά Τυμβίου,350-67-3292,1995-02-13,F,Καρπενήσι,ΤΚ 592 82
3772,73276428,49747,2018-12-15 14:46:43,170000,Refinance existing loans,ACCEPTED,14.0,<NA>,49747,Ευθαλία-Λεμονιά Τυμβίου,350-67-3292,1995-02-13,F,Καρπενήσι,ΤΚ 592 82
9847,30014830,55228,2019-02-16 21:20:10,230000,buying a pet,ACCEPTED,6.7,<NA>,55228,Ηλίας Αραμπατζής,423-19-0023,1971-10-28,M,Λευκάδα,ΤΚ 896 53


In [13]:
loan_customers_visits = pd.merge(loan_customers_ljoin, loan_visits_ljoin, how='left', on=['id_loan','user_id', 'loan_amount', 'loan_purpose', 'outcome', 'interest', 'webvisit_id'])
loan_customers_visits.sort_values('user_id').head()

,id_loan,user_id,timestamp,loan_amount,loan_purpose,outcome,interest,webvisit_id,id_customers,name,ssn,birthday,gender,city,zip_code,timestamp_loan,id_visits,timestamp_visits,referrer,campaign_name
9339,89754689,29817,2018-05-06 20:42:40,290000,Home purchase,ACCEPTED,8.7,<NA>,29817,Νεοκλής Γιαννακέας,539-03-6064,1995-08-10,M,Φλώρινα,17331,NaT,NaN,NaT,NaN,NaN
1667,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,46037,Ελισσαίος Κουτσικόπουλος,219-88-6096,1996-08-05,M,Κιλκίς,ΤΚ 464 00,2017-11-07 07:33:39,54354971.0,2019-01-19 07:51:44,Google,display2
1665,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,46037,Ελισσαίος Κουτσικόπουλος,219-88-6096,1996-08-05,M,Κιλκίς,ΤΚ 464 00,2017-11-07 07:33:39,54354971.0,2018-12-18 14:27:07,Facebook,display3
1666,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,46037,Ελισσαίος Κουτσικόπουλος,219-88-6096,1996-08-05,M,Κιλκίς,ΤΚ 464 00,2017-11-07 07:33:39,54354971.0,2018-11-25 01:07:37,Google,display3
1668,82367130,46037,2017-11-07 07:33:39,90000,Refinance existing loans,ACCEPTED,6.7,54354971,46037,Ελισσαίος Κουτσικόπουλος,219-88-6096,1996-08-05,M,Κιλκίς,ΤΚ 464 00,2017-11-07 07:33:39,54354971.0,2019-06-30 04:35:18,Facebook,display2


### Save the output

In [14]:
loan_customers_visits.to_csv('data_foobank.csv')